# Early Stopping Callback
In this assignment you should write a callback that will check the training loss value at the end of each epoch and if it was less than a threshold it will stop training.

In [3]:
import tensorflow as tf

class EarlyStoppingByLossVal(tf.keras.callbacks.Callback):
    def __init__(self, monitor='loss', value=0.00001, verbose=0):
        super(EarlyStoppingByLossVal, self).__init__()
        self.value = value
    def on_epoch_end(self, epoch, logs=None):
      if logs["loss"] < self.value:
          print("Stopping training...")
          self.model.stop_training = True

## Model Training

In [4]:
import tensorflow_datasets as tfds
ds, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_ds, validation_ds, test_ds) = ds

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteQUNTF2/horses_or_humans-train.tfrecord*..…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteQUNTF2/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [5]:
IMAGE_SIZE = (150, 150)
def preprocess_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

BATCH_SIZE = 32
train_batches = train_ds.shuffle(num_examples // 4).map(preprocess_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_ds.map(preprocess_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_ds.map(preprocess_image).batch(1)

In [6]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [7]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_batches,
          epochs=10,
          validation_data=validation_batches,
          callbacks=[EarlyStoppingByLossVal(value=.5, verbose=1)])

Epoch 1/10
26/26 [==============================] - 21s 726ms/step - loss: 0.6759 - accuracy: 0.5462 - val_loss: 0.6483 - val_accuracy: 0.8537
Epoch 2/10
26/26 [==============================] - 16s 606ms/step - loss: 0.6264 - accuracy: 0.6825 - val_loss: 0.5978 - val_accuracy: 0.6683
Epoch 3/10
26/26 [==============================] - 17s 647ms/step - loss: 0.5756 - accuracy: 0.7238 - val_loss: 0.5288 - val_accuracy: 0.8829
Epoch 4/10
26/26 [==============================] - 17s 616ms/step - loss: 0.5154 - accuracy: 0.7701 - val_loss: 0.5581 - val_accuracy: 0.6244
Epoch 5/10
26/26 [==============================] - 16s 601ms/step - loss: 0.4511 - accuracy: 0.8151 - val_loss: 0.4134 - val_accuracy: 0.8537
